In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import optuna
from xgboost import XGBRegressor
import joblib
from sklearn.multioutput import MultiOutputRegressor
import ast

In [2]:
path = "/Users/amrtamer/Documents/Internship"
# path = "/content/drive/MyDrive/Internship"

In [3]:
data = pd.read_csv(path + r'/data/Volve/Volve_cleaned_prepared.csv')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.101891,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.101891,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.101891,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.113263,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,0.082298,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."


In [4]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{5599: 'NO 15/9-F-12 H',
 7289: 'NO 15/9-F-15 D',
 7405: 'NO 15/9-F-1 C',
 5769: 'NO 15/9-F-5 AH',
 7078: 'NO 15/9-F-11 H',
 5351: 'NO 15/9-F-14 H'}

In [5]:
identification_column = ['well_name', 'WELL_BORE_CODE', 'date']
single_feature_columns = ['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']
multiple_feature_columns = ['Lag_short', 'Lag_medium', 'Lag_long']
feature_columns = single_feature_columns + multiple_feature_columns
target_column = ['oil_rate']

In [6]:
data = data[identification_column + feature_columns]
data.shape

(7987, 26)

In [7]:
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].mean().round(4))
print(data[['prod_hrs', 'bhp', 'bht', 'dp_tubing', 'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'whp', 'wht', 'choke_size_percentage', 'oil_vol', 'gas_vol', 'water_vol', 'linear_regressor', 'EMA_short', 'EMA_medium', 'EMA_long', 'Rolling_short', 'Rolling_medium', 'Rolling_long', 'oil_rate']].std().round(4))

prod_hrs                -0.0
bhp                     -0.0
bht                      0.0
dp_tubing               -0.0
AVG_ANNULUS_PRESS        0.0
AVG_CHOKE_SIZE_P        -0.0
whp                     -0.0
wht                     -0.0
choke_size_percentage    0.0
oil_vol                  0.0
gas_vol                  0.0
water_vol                0.0
linear_regressor        -0.0
EMA_short                0.0
EMA_medium               0.0
EMA_long                -0.0
Rolling_short           -0.0
Rolling_medium           0.0
Rolling_long            -0.0
oil_rate                 0.0
dtype: float64
prod_hrs                 1.0
bhp                      1.0
bht                      1.0
dp_tubing                1.0
AVG_ANNULUS_PRESS        1.0
AVG_CHOKE_SIZE_P         1.0
whp                      1.0
wht                      1.0
choke_size_percentage    1.0
oil_vol                  1.0
gas_vol                  1.0
water_vol                1.0
linear_regressor         1.0
EMA_short                1.0

In [8]:
data

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,oil_rate,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,-0.102310,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.005190,-0.102310,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.005190,0.177706,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.005190,-0.042238,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.005190,0.028674,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,5769,NO 15/9-F-5 AH,2016-08-22,0.268405,-0.880933,-0.806036,-0.814175,1.149266,1.150153,-1.140330,...,-0.692802,-0.702316,-0.715245,-0.692202,-0.703021,-0.724022,-0.683109,"[-0.6852415797773639, -0.6943429268335604, -0....","[-0.6933422169244923, -0.687878486910018, -0.6...","[-0.7133929373653101, -0.7128670168291575, -0...."
7983,5769,NO 15/9-F-5 AH,2016-08-23,0.268405,-0.880933,-0.806036,-0.814175,1.158798,1.150153,-1.136215,...,-0.689514,-0.699464,-0.713067,-0.690182,-0.699482,-0.722275,-0.677835,"[-0.6943429268335604, -0.6840874763785847, -0....","[-0.687878486910018, -0.6852415797773639, -0.6...","[-0.7128670168291575, -0.704496114962062, -0.6..."
7984,5769,NO 15/9-F-5 AH,2016-08-24,0.268405,-0.880933,-0.806036,-0.814175,1.141085,1.150153,-1.137527,...,-0.687737,-0.697357,-0.711216,-0.688940,-0.696914,-0.720063,-0.679069,"[-0.6840874763785847, -0.6831086798251896, -0....","[-0.6852415797773639, -0.6943429268335604, -0....","[-0.704496114962062, -0.6974619277910209, -0.6..."
7985,5769,NO 15/9-F-5 AH,2016-08-25,0.268405,-1.334973,-1.285316,-1.327391,1.140689,1.150153,-1.140776,...,-0.686410,-0.695555,-0.709500,-0.685783,-0.695013,-0.718130,-0.678646,"[-0.6831086798251896, -0.6778348655598816, -0....","[-0.6943429268335604, -0.6840874763785847, -0....","[-0.6974619277910209, -0.6908660077334403, -0...."


In [9]:
data.isna().any()

well_name                False
WELL_BORE_CODE           False
date                     False
prod_hrs                 False
bhp                      False
bht                      False
dp_tubing                False
AVG_ANNULUS_PRESS        False
AVG_CHOKE_SIZE_P         False
whp                      False
wht                      False
choke_size_percentage    False
oil_vol                  False
gas_vol                  False
water_vol                False
linear_regressor         False
EMA_short                False
EMA_medium               False
EMA_long                 False
Rolling_short            False
Rolling_medium           False
Rolling_long             False
oil_rate                 False
Lag_short                False
Lag_medium               False
Lag_long                 False
dtype: bool

In [10]:
for column in multiple_feature_columns:
    try:
        data[column] = data[column].apply(ast.literal_eval)
    except Exception:
        pass

In [11]:
days_window = 5
predictions_days = 14
training_gap = predictions_days + days_window

In [79]:
sequences = []
for well_name in data['well_name'].unique():
    well_data = data[data['well_name'] == well_name].sort_values(by='date').reset_index()
    for i in range(len(well_data) - training_gap + 1):
        single_features = np.array(well_data.iloc[i:i + days_window][single_feature_columns])
        multiple_features = np.array(well_data.iloc[i:i + days_window][multiple_feature_columns])
        multiple_flattened = []
        for k in multiple_features:
           flattened_list = [item for sublist in k for item in sublist]
           multiple_flattened.append(flattened_list)
        multiple_flattened = np.array(multiple_flattened)
        inputs = np.concatenate((single_features, multiple_flattened), axis=1)
        targets = well_data.iloc[i + days_window:i + days_window + predictions_days][target_column].values
        inputs_dates = list(well_data.iloc[i:i + days_window]['date'])
        targets_dates = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['date'])
        inputs_index = list(well_data.iloc[i:i + days_window]['index'])
        targets_index = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['index'])
        sequences.append((well_name, inputs_dates, targets_dates, inputs_index, targets_index, inputs, targets))
        if len(well_data.iloc[i:i + days_window + predictions_days]['well_name'].unique()) > 1:
          print('violation')

dataset = pd.DataFrame(sequences, columns=['well_name', 'inputs_dates', 'targets_dates', 'inputs_index', 'targets_index', 'inputs', 'targets', ])
dataset

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,7405,"[2014-04-22, 2014-04-23, 2014-04-24, 2014-04-2...","[2014-04-27, 2014-04-28, 2014-04-29, 2014-04-3...","[0, 1, 2, 3, 4]","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[[0.2684054414666464, 1.029311137344964, 0.605...","[[0.0314347175089187], [0.0285494590119704], [..."
1,7405,"[2014-04-23, 2014-04-24, 2014-04-25, 2014-04-2...","[2014-04-28, 2014-04-29, 2014-04-30, 2014-05-0...","[1, 2, 3, 4, 5]","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[[0.2684054414666464, 0.8457950056169782, 0.63...","[[0.0285494590119704], [-0.0197694902470496], ..."
2,7405,"[2014-04-24, 2014-04-25, 2014-04-26, 2014-04-2...","[2014-04-29, 2014-04-30, 2014-05-01, 2014-05-0...","[2, 3, 4, 5, 6]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, ...","[[0.2684054414666464, 0.7750251338314447, 0.63...","[[-0.0197694902470496], [-0.0590966592282385],..."
3,7405,"[2014-04-25, 2014-04-26, 2014-04-27, 2014-04-2...","[2014-04-30, 2014-05-01, 2014-05-02, 2014-05-0...","[3, 4, 5, 6, 7]","[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...","[[0.2684054414666464, 0.7050330190704033, 0.64...","[[-0.0590966592282385], [-0.068387922033601], ..."
4,7405,"[2014-04-26, 2014-04-27, 2014-04-28, 2014-04-2...","[2014-05-01, 2014-05-02, 2014-05-03, 2014-05-0...","[4, 5, 6, 7, 8]","[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20...","[[0.2684054414666464, 0.6253569546426345, 0.64...","[[-0.068387922033601], [-0.1698175409921428], ..."
...,...,...,...,...,...,...,...
7874,5769,"[2016-08-04, 2016-08-05, 2016-08-06, 2016-08-0...","[2016-08-09, 2016-08-10, 2016-08-11, 2016-08-1...","[7964, 7965, 7966, 7967, 7968]","[7969, 7970, 7971, 7972, 7973, 7974, 7975, 797...","[[0.2684054414666464, -0.4755216279671733, -0....","[[-0.7318767962210462], [-0.7237141278873048],..."
7875,5769,"[2016-08-05, 2016-08-06, 2016-08-07, 2016-08-0...","[2016-08-10, 2016-08-11, 2016-08-12, 2016-08-1...","[7965, 7966, 7967, 7968, 7969]","[7970, 7971, 7972, 7973, 7974, 7975, 7976, 797...","[[0.2684054414666464, -0.4755216279671733, -0....","[[-0.7237141278873048], [-0.7192072810705527],..."
7876,5769,"[2016-08-06, 2016-08-07, 2016-08-08, 2016-08-0...","[2016-08-11, 2016-08-12, 2016-08-13, 2016-08-1...","[7966, 7967, 7968, 7969, 7970]","[7971, 7972, 7973, 7974, 7975, 7976, 7977, 797...","[[0.2684054414666464, -0.4755216279671733, -0....","[[-0.7192072810705527], [-0.7133929373653101],..."
7877,5769,"[2016-08-07, 2016-08-08, 2016-08-09, 2016-08-1...","[2016-08-12, 2016-08-13, 2016-08-14, 2016-08-1...","[7967, 7968, 7969, 7970, 7971]","[7972, 7973, 7974, 7975, 7976, 7977, 7978, 797...","[[0.2684054414666464, -0.4755216279671733, -0....","[[-0.7133929373653101], [-0.7128670168291575],..."


In [80]:
check = True
for i, j in well_info.items():
    if len(dataset[dataset['well_name'] == i]) < 10:
        print("The input/output gap is too high for well", i)
        check = False
if check:
    print("The input/output gap is fine")

The input/output gap is fine


In [81]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [82]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [83]:
# train_wells = []
# test_wells = []
# for i in well_info.keys():
#     well = dataset[dataset['well_name'] == i]
#     train_set_length = int((train_size + val_size) * len(well))
#     train_wells.append(well[:train_set_length + 1])
#     test_set_length = int(test_size * len(well))
#     test_wells.append(well[-test_set_length:])

# # Flatten the lists of DataFrames
# temp_data = pd.concat(train_wells).reset_index(drop=True)
# data_test = pd.concat(test_wells).reset_index(drop=True)


# data_train, data_val = train_test_split(temp_data, test_size=test_size_param2, stratify=temp_data['well_name'], shuffle=True)

# data_train.reset_index(drop=True, inplace=True)
# data_val.reset_index(drop=True, inplace=True)
# data_test.reset_index(drop=True, inplace=True)

# data_train.to_pickle(path + "/data/Volve/data_train.pkl")
# data_val.to_pickle(path + "/data/Volve/data_val.pkl")
# data_test.to_pickle(path + "/data/Volve/data_test.pkl")

((4730, 7), (1577, 7), (1573, 7))

In [12]:
data_train = pd.read_pickle(path + "/data/Volve/data_train.pkl")
data_val = pd.read_pickle(path + "/data/Volve/data_val.pkl")
data_test = pd.read_pickle(path + "/data/Volve/data_test.pkl")

In [13]:
data_train['inputs'][0].shape, data_train['targets'][0].shape

((5, 38), (14, 1))

In [14]:
check=True
for i in [data_train, data_val, data_test]:
    for j in well_info.keys():
        if j not in set(list(i['well_name'])):
            print("Some datasets do not have specific well")
            check = False
            break
if check:
    print("All datasets have all wells")

All datasets have all wells


In [15]:
data_train

,well_name,inputs_dates,targets_dates,inputs_index,targets_index,inputs,targets
0,5599,"[2014-04-23, 2014-04-24, 2014-04-25, 2014-04-2...","[2014-04-28, 2014-04-29, 2014-04-30, 2014-05-0...","[3594, 3595, 3596, 3597, 3598]","[3599, 3600, 3601, 3602, 3603, 3604, 3605, 360...","[[0.2684054414666464, -1.739674518873222, -1.7...","[[-0.7990303313354917], [-0.8065539167832304],..."
1,5351,"[2010-11-28, 2010-11-29, 2010-11-30, 2010-12-0...","[2010-12-03, 2010-12-04, 2010-12-05, 2010-12-0...","[5198, 5199, 5200, 5201, 5202]","[5203, 5204, 5205, 5206, 5207, 5208, 5209, 521...","[[0.2684054414666464, 0.5661170003239738, 0.59...","[[0.5549886112488323], [0.5461210066531479], [..."
2,5351,"[2008-08-19, 2008-08-20, 2008-08-21, 2008-08-3...","[2008-09-03, 2008-09-04, 2008-09-05, 2008-09-0...","[4416, 4417, 4418, 4419, 4420]","[4421, 4422, 4423, 4424, 4425, 4426, 4427, 442...","[[0.2684054414666464, 0.5504442834987169, 0.56...","[[0.6269082445677003], [0.6429488209203544], [..."
3,7078,"[2014-04-10, 2014-04-11, 2014-04-12, 2014-04-1...","[2014-04-15, 2014-04-16, 2014-04-17, 2014-04-1...","[675, 676, 677, 678, 679]","[680, 681, 682, 683, 684, 685, 686, 687, 688, ...","[[-0.3380783807804748, 0.5960973844865403, 0.5...","[[-0.0361972025499273], [-0.042033459610843], ..."
4,5351,"[2012-02-20, 2012-02-21, 2012-02-22, 2012-02-2...","[2012-02-25, 2012-02-26, 2012-02-27, 2012-02-2...","[5575, 5576, 5577, 5578, 5579]","[5580, 5581, 5582, 5583, 5584, 5585, 5586, 558...","[[0.2684054414666464, 0.3550862202072024, 0.57...","[[-0.0410035318942108], [-0.0421211130335352],..."
...,...,...,...,...,...,...,...
4725,7078,"[2014-12-03, 2014-12-04, 2014-12-05, 2014-12-0...","[2014-12-08, 2014-12-09, 2014-12-10, 2014-12-1...","[907, 908, 909, 910, 911]","[912, 913, 914, 915, 916, 917, 918, 919, 920, ...","[[0.2684054414666464, 0.9655680167542784, 0.57...","[[-0.8064516544567563], [-0.8021931590042984],..."
4726,5599,"[2008-12-07, 2008-12-08, 2008-12-09, 2008-12-1...","[2008-12-12, 2008-12-13, 2008-12-14, 2008-12-1...","[1839, 1840, 1841, 1842, 1843]","[1844, 1845, 1846, 1847, 1848, 1849, 1850, 185...","[[0.2684054414666464, 0.4771871507005275, 0.61...","[[2.8730491144084365], [2.903764334610126], [2..."
4727,5599,"[2009-12-15, 2009-12-16, 2009-12-17, 2009-12-1...","[2009-12-20, 2009-12-21, 2009-12-22, 2009-12-2...","[2193, 2194, 2195, 2196, 2197]","[2198, 2199, 2200, 2201, 2202, 2203, 2204, 220...","[[0.2684054414666464, 0.6730763535118753, 0.62...","[[2.72610603129202], [2.951484318814084], [2.9..."
4728,5599,"[2011-11-19, 2011-11-20, 2011-11-21, 2011-11-2...","[2011-11-24, 2011-11-28, 2011-12-02, 2011-12-0...","[2830, 2831, 2832, 2833, 2834]","[2835, 2836, 2837, 2838, 2839, 2840, 2841, 284...","[[0.2684054414666464, -1.739674518873222, -1.7...","[[-0.1699037008527869], [-0.3414906628596778],..."


In [16]:
# Split sequences into input features and targets
X_train = np.array([i for i in data_train['inputs']])
Y_train = np.array([i for i in data_train['targets']]).squeeze()

X_val = np.array([i for i in data_val['inputs']])
Y_val = np.array([i for i in data_val['targets']]).squeeze()

X_test = np.array([i for i in data_test['inputs']])
Y_test = np.array([i for i in data_test['targets']]).squeeze()

X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

((4730, 190), (4730, 14), (1577, 190), (1577, 14), (1573, 190), (1573, 14))

In [88]:
class XGBoost():
    def __init__(self, n_estimators, learning_rate, max_depth, min_child_weight, subsample, colsample_bytree, output_length, early_stopping_rounds):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.output_length = output_length        
        self.early_stopping_rounds = early_stopping_rounds
        self.xgboost_model = []
        for i in range(self.output_length):
            model = XGBRegressor(objective='reg:squarederror', n_estimators=self.n_estimators, learning_rate=self.learning_rate, max_depth=self.max_depth, min_child_weight=self.min_child_weight, subsample=self.subsample, colsample_bytree=self.colsample_bytree, early_stopping_rounds=self.early_stopping_rounds)
            self.xgboost_model.append(model)

    def fit(self, X_train, Y_train, X_val, Y_val):
        for i in range(self.output_length):
            self.xgboost_model[i].fit(X_train, Y_train[:, i].reshape(-1, 1), eval_set=[(X_val, Y_val[:, i].reshape(-1, 1))], verbose=True)
    
    def predict(self, X):
        outputs = np.zeros((X.shape[0], self.output_length))
        for i, model in enumerate(self.xgboost_model):
            outputs[:, i] = model.predict(X)
        return outputs

    def save_model(self, path):
        new_path = os.path.join(path, "XGBoost")
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        for i, model in enumerate(self.xgboost_model):
            model.save_model(os.path.join(new_path, f"XGBoost_{i}.json"))  # Save the XGBoost model
    
    def load_model(self, path):
        new_path = os.path.join(path, "XGBoost")
        for i, model in enumerate(self.xgboost_model):
            model.load_model(new_path + f"/XGBoost_{i}.json")

In [17]:
def calculate_metrics(y_true, y_pred):
    # Calculate metrics for the predictions
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred, multioutput='uniform_average')
    loss = mean_squared_error(y_true, y_pred)
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'Loss': loss}

def evaluate(mode, model):
    X_modes = {'train': X_train, 'val': X_val, 'test': X_test}
    Y_modes = {'train': Y_train, 'val': Y_val, 'test': Y_test}
    X = X_modes[mode]
    Y = Y_modes[mode].reshape(-1)
    outputs = model.predict(X).reshape(-1)
    # Calculate metrics
    metrics = calculate_metrics(Y, outputs)
    print(f"Loss is {metrics['Loss']:.4f}")
    print(f"MAE is {metrics['MAE']:.4f}")
    print(f"RMSE is {metrics['RMSE']:.4f}")
    print(f"R² is {metrics['R2']:.4f}")
    print("\n")
    return metrics

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    lr = trial.suggest_float('lr', 1e-4, 0.4, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 50)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    xgb = XGBRegressor(output_length= predictions_days, n_estimators=n_estimators, learning_rate=lr, max_depth=max_depth, min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree)
    model = MultiOutputRegressor(xgb)
    model.fit(X_train, Y_train)
    val_preds = model.predict(X_val)
    val_loss = mean_squared_error(Y_val, val_preds)
    return val_loss

In [92]:
%%capture
# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
best_params = study.best_params
n_estimators = best_params['n_estimators']
lr = best_params['lr']
max_depth = best_params['max_depth']
min_child_weight = best_params['min_child_weight']
subsample = best_params['subsample']
colsample_bytree = best_params['colsample_bytree']

[I 2024-07-02 14:53:20,150] A new study created in memory with name: no-name-0dbfd818-6c65-4881-a15f-f42b6644bd43
[I 2024-07-02 14:55:04,966] Trial 0 finished with value: 0.029965498943533227 and parameters: {'n_estimators': 806, 'lr': 0.0055242708973689885, 'max_depth': 23, 'min_child_weight': 34, 'subsample': 0.9276311307669547, 'colsample_bytree': 0.8895332353188312}. Best is trial 0 with value: 0.029965498943533227.
[I 2024-07-02 14:56:33,209] Trial 1 finished with value: 0.6925375320870095 and parameters: {'n_estimators': 559, 'lr': 0.0004101348293390399, 'max_depth': 14, 'min_child_weight': 3, 'subsample': 0.7218004340618807, 'colsample_bytree': 0.6437446549615271}. Best is trial 0 with value: 0.029965498943533227.
[I 2024-07-02 14:58:39,561] Trial 2 finished with value: 0.02355662488090007 and parameters: {'n_estimators': 752, 'lr': 0.021883664323391568, 'max_depth': 30, 'min_child_weight': 16, 'subsample': 0.5216476775900352, 'colsample_bytree': 0.8877023913280608}. Best is tri

In [93]:
print('Best hyperparameters:', best_params)

Best hyperparameters: {'n_estimators': 594, 'lr': 0.045570993763519016, 'max_depth': 17, 'min_child_weight': 12, 'subsample': 0.7433625471337193, 'colsample_bytree': 0.5992551725398402}


In [18]:
n_estimators = 594
lr = 0.045570993763519016 
max_depth = 17
min_child_weight = 12
subsample = 0.7433625471337193 
colsample_bytree = 0.5992551725398402

In [19]:
# Create and configure the XGBoost regressor with hyperparameters
xgb = XGBRegressor(n_estimators=n_estimators, learning_rate=lr, max_depth=max_depth, min_child_weight=min_child_weight, subsample=subsample, colsample_bytree=colsample_bytree) 

# Wrap the XGBoost regressor in a MultiOutputRegressor
model = MultiOutputRegressor(xgb)
model.fit(X_train, Y_train)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5992551725398402,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.045570993763519016,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=17,
                                            max_leaves=None,
                                            min_child_weight=12, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=594, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))

In [20]:
model.predict(X_test).shape

(1573, 14)

In [22]:
train_metrics = evaluate('train', model)
val_sizemetrics = evaluate('val', model)
test_metrics = evaluate('test', model)

Loss is 0.0002
MAE is 0.0059
RMSE is 0.0123
R² is 0.9999


Loss is 0.0232
MAE is 0.0764
RMSE is 0.1523
R² is 0.9778


Loss is 0.0043
MAE is 0.0481
RMSE is 0.0656
R² is 0.8202




In [28]:
train_metrics = evaluate('train', model)
val_sizemetrics = evaluate('val', model)
test_metrics = evaluate('test', model)

Loss is 0.0003
MAE is 0.0095
RMSE is 0.0187
R² is 0.9997


Loss is 0.0284
MAE is 0.0835
RMSE is 0.1686
R² is 0.9728


Loss is 0.0066
MAE is 0.0638
RMSE is 0.0814
R² is -0.9787




In [25]:
all_data = pd.read_pickle(path + "/data/Volve/all_data.pkl")

1. add time elapsed column
2. find a way to fill gaps
5. deal with wht and whp
6. graphs 50 pdf
hyperparameter tuning
look into auto hyperparameter fintuning

In [ ]:
# Count occurrences of each unique value
value_counts = data_test['well_name'].value_counts()

# Calculate percentages
percentages = (value_counts / len(data_test)) * 100

# Print the percentages
print(percentages)